# Part I - Exploring Prosper Loan Data
## by Israel Ogunmola

## Introduction
---

If you want to consolidate debt, finance a large purchase, or cover an emergency expense, a personal loan can be immensely useful. However, many personal loan lenders require borrowers to have good or excellent credit, making it difficult to qualify for a loan. Borrowers with credit ratings in the fair range or below may have better chances of obtaining loans at a better rate by working with a peer-to-peer lender.

[Prosper is a personal loan pioneer](https://www.prosper.com/) — the company became the first firm to enter the peer-to-peer lending arena when it launched in 2005. Since then, the platfom has originated more than 20 billion USD in personal loans by matching over 1,170,000 borrowers to potential investors through its online platform. Prosper offers unsecured personal loans to customers who have a minimum credit score of 640. It also provides home equity lines of credit (HELOCs).

Our goal is to explore a sample of Prosper loan data to uncover borrower motivations when applying for loans, and identify several factors that may influence loan outcomes.

## Importing Libraries
---
 
A great way to start is by importing the libraries and packages we need. We will import the **Numpy** and **Pandas** libraries to help us load and perform quick, vectorized operations on our data, then the **Matplotlib** and **Seaborn** libraries to help us build informing visuals:

In [1]:
# Data analysis and visualization packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

# Configure visualization behaviours
%matplotlib inline
# %config InlineBackend.figure_format = "retina"

## Preliminary Wrangling
---